In [2]:
# new test with updated database and classifier using different processing method

import sys
sys.path.append("/Users/david/Documents/mastersCode/ubm")
print(sys.path)

['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/Users/david/Documents/mastersCode/ubm/classifiers/hmm-pmc/development', '/Users/david/.conda/envs/ubm/lib/python310.zip', '/Users/david/.conda/envs/ubm/lib/python3.10', '/Users/david/.conda/envs/ubm/lib/python3.10/lib-dynload', '', '/Users/david/.conda/envs/ubm/lib/python3.10/site-packages', '/Users/david/Documents/mastersCode/ubm', '/Users/david/Documents/mastersCode/ubm']


In [3]:
from importlib import reload
import audio_datastore.audio_datastore as myads
reload(myads)
from importlib import reload
from collections import Counter
import pickle
import time

Using cache found in /Users/david/.cache/torch/hub/snakers4_silero-vad_master
Using cache found in /Users/david/.cache/torch/hub/snakers4_silero-vad_master


In [4]:
from data.pitch_tracking_database.load_data import *
speech_pitch_tracking_db = pickle.load(open(normalised_30,'rb'))
ads = speech_pitch_tracking_db['ads']

In [5]:
# split ads
developmentLabels = ["M03","M04","M05","M06","M07","M08","M09","M10","F05","F06","F07","F08","F09","F10",
                     "F03","F04"]
evaluationLabels = ["M01","M02","F01","F02"]
adsTrain = myads.subset(ads, developmentLabels)
adsEvaluate = myads.subset(ads, evaluationLabels)
numFilesPerSpeakerForEnrollment = 4
adsEnroll, adsDet = myads.split(adsEvaluate, numFilesPerSpeakerForEnrollment)
adsTest = adsDet
# adsDet, adsTest = myads.split(adsDet, 2)
# adsTrain.info('Train')
# adsEnroll.info('Enroll')
# adsTest.info('Test')

In [ ]:
from classifiers.classifier_base import ClassifierBase
from classifiers.hmm_pmc.classifier_hmm_pmc import ClassifierHMMPMC
from my_torch.tuts2.torch_transforms import ComposeTransform

def test_classifiers(ads_train, ads_test, classifiers: [ClassifierHMMPMC], verbose=False):
    index = 1
    if verbose:
        print('running all')

    for classifier in classifiers:
        if verbose:
            print('\n')
            print('test #', index)
        print(classifier.info)

        classifier.train(ads_train)
        classifier.train_noise_hmm()
        classifier.adapt_speaker_models()
        classifier.test_all(ads_test)

        index = index + 1
        if verbose:
            print('\n')